# MVP - Engenharia de Dados
## Autor: Marcel Antonio Lopes Pereira


##  Camada Silver

Neste notebook, será continuado o processo de transformação dos dados, avançando para a construção da camada silver do nosso pipeline de dados. A camada silver tem como objetivo realizar um processamento mais refinado dos dados provenientes da camada bronze, aplicando transformações e limpezas essenciais para garantir que os dados estejam padronizados e prontos para análises mais complexas na camada gold.

Utilizando as tabelas da camada bronze como base, foram realizadas operações como o tratamento de nomes de seleções através da tabela former_names, além de aplicar ajustes nos formatos de dados (como conversões de tipos de variáveis) e, quando necessário, a remoção de dados nulos. As tabelas resultantes da camada silver servirão como um ponto de partida para análises mais profundas e criação de métricas na camada gold.

A seguir, serão processados os principais dados de resultados de partidas, disputas de pênaltis, marcadores de gols e o ranking FIFA para garantir que todas as informações estejam corretamente estruturadas e padronizadas. Ao final deste notebook, haverá tabelas com informações prontas para serem utilizadas em análises avançadas, como a performance das seleções ao longo do tempo.
Primeiramente, são executados scripts de criação do database silver.

In [0]:
%sql DROP DATABASE IF EXISTS silver CASCADE

In [0]:
%sql CREATE DATABASE silver

A partir disso, são criadas as tabelas na camada silver a partir das tabelas da camada Bronze. O processo de criação de cada uma delas é descrito a seguir.

### Tabela Results
A primeira tabela a ser criada é a tabela silver.results. 
O fluxo de construção da tabela silver.results inicia com os dados brutos da camada Bronze (bronze.results) e aplica uma série de transformações essenciais para garantir consistência, qualidade e enriquecimento dos dados.

- Padronização de Datas: Converte date para o tipo DATE para garantir consistência em análises temporais.

- Unificação de Nomes de Times:  Usa bronze.former_names para padronizar nomes históricos (ex: "Alemanha Ocidental" → "Alemanha"); Mantém o nome original se não houver correspondência (COALESCE).

- Conversão de Tipos de Dados: Converte home_score e away_score para INT (evitando decimais desnecessários); Transforma neutral em BOOLEAN (TRUE/FALSE).

- Limpeza Básica: Remove registros com placares nulos (WHERE home_score IS NOT NULL); Aplica TRIM() em campos de texto para eliminar espaços extras.

In [0]:
%sql
CREATE OR REPLACE TABLE silver.results AS
SELECT 
  CONCAT(
    CAST(date(r.date) AS STRING), 
    '_', 
    COALESCE(fn_home.current, r.home_team), 
    '_', 
    COALESCE(fn_away.current, r.away_team), 
    '_', 
    CAST(r.home_score AS STRING), 
    '_', 
    CAST(r.away_score AS STRING)
  ) AS match_id,  -- Geração do match_id concatenando as colunas relevantes
  date(r.date) AS match_date,
  COALESCE(fn_home.current, r.home_team) AS home_team,
  COALESCE(fn_away.current, r.away_team) AS away_team,
  CAST(r.home_score AS INT) AS home_score,
  CAST(r.away_score AS INT) AS away_score,
  trim(r.tournament) AS tournament,  -- Apenas remove espaços, sem categorização
  r.city,
  r.country,
  CAST(r.neutral AS BOOLEAN) AS is_neutral
FROM bronze.results r
LEFT JOIN bronze.former_names fn_home ON r.home_team = fn_home.former
LEFT JOIN bronze.former_names fn_away ON r.away_team = fn_away.former
WHERE r.home_score IS NOT NULL;


num_affected_rows,num_inserted_rows


São incluídos comentários descrevendo a tabela e seus atributos.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.results 
SET TBLPROPERTIES (
  'comment' = 'Tabela de resultados de partidas de futebol internacionais. Contém dados brutos padronizados para uso na camada Gold. Fonte: bronze.results.'
);

-- Comentários das colunas (um comando por coluna)
ALTER TABLE silver.results CHANGE COLUMN match_date match_date DATE COMMENT 'Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 (primeiro jogo internacional) e a data atual.';
ALTER TABLE silver.results CHANGE COLUMN home_team home_team STRING COMMENT 'Nome do time da casa padronizado. Domínio: Valores da tabela silver.teams (ex: "Brazil", "Germany").';
ALTER TABLE silver.results CHANGE COLUMN away_team away_team STRING COMMENT 'Nome do time visitante padronizado. Domínio: Valores da tabela silver.teams.';
ALTER TABLE silver.results CHANGE COLUMN home_score home_score INT COMMENT 'Gols marcados pelo time da casa. Domínio: Inteiro ≥ 0 (valores nulos substituídos por 0).';
ALTER TABLE silver.results CHANGE COLUMN away_score away_score INT COMMENT 'Gols marcados pelo time visitante. Domínio: Inteiro ≥ 0 (valores nulos substituídos por 0).';
ALTER TABLE silver.results CHANGE COLUMN tournament tournament STRING COMMENT 'Nome do torneio sem formatação adicional. Domínio: Texto livre (ex: "FIFA World Cup", "Copa América").';
ALTER TABLE silver.results CHANGE COLUMN city city STRING COMMENT 'Cidade onde a partida foi realizada. Domínio: Texto livre (ex: "Rio de Janeiro", "Berlin").';
ALTER TABLE silver.results CHANGE COLUMN country country STRING COMMENT 'País onde a partida foi realizada. Domínio: Nomes de países ISO 3166 (ex: "Brazil", "Spain").';
ALTER TABLE silver.results CHANGE COLUMN is_neutral is_neutral BOOLEAN COMMENT 'Indica se o jogo foi em local neutro. Domínio: TRUE/FALSE.';

In [0]:
%sql
DESCRIBE TABLE EXTENDED silver.results;

col_name,data_type,comment
match_id,string,null
match_date,date,Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 (primeiro jogo internacional) e a data atual.
home_team,string,"Nome do time da casa padronizado. Domínio: Valores da tabela silver.teams (ex: ""Brazil"", ""Germany"")."
away_team,string,Nome do time visitante padronizado. Domínio: Valores da tabela silver.teams.
home_score,int,Gols marcados pelo time da casa. Domínio: Inteiro ≥ 0 (valores nulos substituídos por 0).
away_score,int,Gols marcados pelo time visitante. Domínio: Inteiro ≥ 0 (valores nulos substituídos por 0).
tournament,string,"Nome do torneio sem formatação adicional. Domínio: Texto livre (ex: ""FIFA World Cup"", ""Copa América"")."
city,string,"Cidade onde a partida foi realizada. Domínio: Texto livre (ex: ""Rio de Janeiro"", ""Berlin"")."
country,string,"País onde a partida foi realizada. Domínio: Nomes de países ISO 3166 (ex: ""Brazil"", ""Spain"")."
is_neutral,boolean,Indica se o jogo foi em local neutro. Domínio: TRUE/FALSE.


É verificada a criação da tabela.

In [0]:
%sql
-- Verificar a tabela silver.results
SELECT * FROM silver.results LIMIT 20;

match_id,match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral
1872-11-30_Scotland_England_0_0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,false
1873-03-08_England_Scotland_4_2,1873-03-08,England,Scotland,4,2,Friendly,London,England,false
1874-03-07_Scotland_England_2_1,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,false
1875-03-06_England_Scotland_2_2,1875-03-06,England,Scotland,2,2,Friendly,London,England,false
1876-03-04_Scotland_England_3_0,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,false
1876-03-25_Scotland_Wales_4_0,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,false
1877-03-03_England_Scotland_1_3,1877-03-03,England,Scotland,1,3,Friendly,London,England,false
1877-03-05_Wales_Scotland_0_2,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,false
1878-03-02_Scotland_England_7_2,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,false
1878-03-23_Scotland_Wales_9_0,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,false


In [0]:
%sql
SELECT *
FROM silver.results
WHERE tournament = "Copa América"

match_id,match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral
1916-07-02_Chile_Uruguay_0_4,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,true
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Chile,6,1,Copa América,Buenos Aires,Argentina,false
1916-07-08_Brazil_Chile_1_1,1916-07-08,Brazil,Chile,1,1,Copa América,Buenos Aires,Argentina,true
1916-07-10_Argentina_Brazil_1_1,1916-07-10,Argentina,Brazil,1,1,Copa América,Buenos Aires,Argentina,false
1916-07-12_Brazil_Uruguay_1_2,1916-07-12,Brazil,Uruguay,1,2,Copa América,Buenos Aires,Argentina,true
1916-07-17_Argentina_Uruguay_0_0,1916-07-17,Argentina,Uruguay,0,0,Copa América,Avellaneda,Argentina,false
1917-09-30_Uruguay_Chile_4_0,1917-09-30,Uruguay,Chile,4,0,Copa América,Montevideo,Uruguay,false
1917-10-03_Argentina_Brazil_4_2,1917-10-03,Argentina,Brazil,4,2,Copa América,Montevideo,Uruguay,true
1917-10-06_Argentina_Chile_1_0,1917-10-06,Argentina,Chile,1,0,Copa América,Montevideo,Uruguay,true
1917-10-07_Uruguay_Brazil_4_0,1917-10-07,Uruguay,Brazil,4,0,Copa América,Montevideo,Uruguay,false


Não foram verificadas inconsistências nessa condição, logo não é necessária correção. A seguir, são verificadas outras condições e caso seja indentificada alguma inconsitência, o script de correção é utilizado e a tabela atualizada.

### Tabela Shootouts
O fluxo de construção da tabela shootouts na camada Silver segue uma sequência lógica de transformações a partir dos dados brutos:

- Padronização de Datas: Converte date para DATE.

- Unificação de Nomes de Times: Padroniza home_team, away_team e winner usando bronze.former_names.

- Preservação de Dados Originais: Mantém first_shooter (time que bateu primeiro) sem modificações.

- Filtro de Qualidade: Não remove registros (todos os pênaltis são considerados válidos).

In [0]:
%sql
CREATE OR REPLACE TABLE silver.shootouts AS
SELECT 
  r.match_id,  -- Usando o match_id da tabela results já existente
  date(s.date) AS match_date,
  COALESCE(fn_home.current, s.home_team) AS home_team,
  COALESCE(fn_away.current, s.away_team) AS away_team,
  COALESCE(fn_winner.current, s.winner) AS winner,
  s.first_shooter
FROM bronze.shootouts s
JOIN silver.results r
  ON r.home_team = s.home_team
  AND r.away_team = s.away_team
  AND r.match_date = date(s.date)
LEFT JOIN bronze.former_names fn_home ON s.home_team = fn_home.former
LEFT JOIN bronze.former_names fn_away ON s.away_team = fn_away.former
LEFT JOIN bronze.former_names fn_winner ON s.winner = fn_winner.former;


num_affected_rows,num_inserted_rows


O mesmo procedimento de inserção de comentários é utilizado.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.shootouts 
SET TBLPROPERTIES (
  'comment' = 'Registro de disputas de pênaltis em partidas de futebol. Fonte: bronze.shootouts.'
);

-- Comentários das colunas
ALTER TABLE silver.shootouts CHANGE COLUMN match_date match_date DATE COMMENT 'Data da partida associada à disputa de pênaltis. Domínio: Deve existir em silver.results.match_date.';
ALTER TABLE silver.shootouts CHANGE COLUMN home_team home_team STRING COMMENT 'Time da casa padronizado. Domínio: Valores de silver.results.home_team.';
ALTER TABLE silver.shootouts CHANGE COLUMN away_team away_team STRING COMMENT 'Time visitante padronizado. Domínio: Valores de silver.results.away_team.';
ALTER TABLE silver.shootouts CHANGE COLUMN winner winner STRING COMMENT 'Time vencedor da disputa de pênaltis. Domínio: Valores de home_team ou away_team.';
ALTER TABLE silver.shootouts CHANGE COLUMN first_shooter first_shooter STRING COMMENT 'Time que bateu primeiro nos pênaltis. Domínio: Valores de home_team ou away_team.';

In [0]:
%sql
DESCRIBE TABLE silver.shootouts;

col_name,data_type,comment
match_id,string,null
match_date,date,Data da partida associada à disputa de pênaltis. Domínio: Deve existir em silver.results.match_date.
home_team,string,Time da casa padronizado. Domínio: Valores de silver.results.home_team.
away_team,string,Time visitante padronizado. Domínio: Valores de silver.results.away_team.
winner,string,Time vencedor da disputa de pênaltis. Domínio: Valores de home_team ou away_team.
first_shooter,string,Time que bateu primeiro nos pênaltis. Domínio: Valores de home_team ou away_team.


É verificada a criação da tabela.

In [0]:
%sql
-- Verificar a tabela silver.shootouts
SELECT * FROM silver.shootouts LIMIT 20;

match_id,match_date,home_team,away_team,winner,first_shooter
1967-08-22_India_Taiwan_1_1,1967-08-22,India,Taiwan,Taiwan,null
1971-11-14_South Korea_Vietnam Republic_1_1,1971-11-14,South Korea,Vietnam Republic,South Korea,null
1972-05-07_South Korea_Iraq_0_0,1972-05-07,South Korea,Iraq,Iraq,null
1972-05-17_Thailand_South Korea_1_1,1972-05-17,Thailand,South Korea,South Korea,null
1972-05-19_Thailand_Cambodia_2_2,1972-05-19,Thailand,Cambodia,Thailand,null
1973-04-21_Senegal_Ghana_1_0,1973-04-21,Senegal,Ghana,Ghana,null
1973-06-14_Guinea_Mali_1_1,1973-06-14,Guinea,Mali,Guinea,null
1973-06-14_Mauritius_Tanzania_1_1,1973-06-14,Mauritius,Tanzania,Mauritius,null
1973-07-26_Cambodia_Singapore_1_1,1973-07-26,Cambodia,Singapore,Singapore,null
1973-07-26_Malaysia_Kuwait_0_0,1973-07-26,Malaysia,Kuwait,Malaysia,null


### Tabela Goalscorers
O fluxo de transformação para criação da tabela silver.goalscorers é descrito a seguir:

- Padronização de Datas: Converte date para DATE.

- Unificação de Nomes de Times e Jogadores: Padroniza scoring_team, home_team e away_team via bronze.former_names; Não formata scorer (isso será feito na camada Gold).

- Conversão de Flags Booleanas: Transforma own_goal e penalty em BOOLEAN (TRUE/FALSE).

- Filtro de Qualidade: Remove gols sem jogador associado (WHERE scorer IS NOT NULL).

In [0]:
%sql
CREATE OR REPLACE TABLE silver.goalscorers AS
SELECT 
  r.match_id,  -- Usando o match_id da tabela results já existente
  date(g.date) AS match_date,
  COALESCE(fn_team.current, g.team) AS scoring_team,
  COALESCE(fn_home.current, g.home_team) AS home_team,
  COALESCE(fn_away.current, g.away_team) AS away_team,
  g.scorer AS player_name_raw,  -- Sem formatação
  CAST(g.own_goal AS BOOLEAN) AS is_own_goal,
  CAST(g.penalty AS BOOLEAN) AS is_penalty
FROM bronze.goalscorers g
JOIN silver.results r
  ON r.home_team = g.home_team
  AND r.away_team = g.away_team
  AND r.match_date = date(g.date)
LEFT JOIN bronze.former_names fn_team ON g.team = fn_team.former
LEFT JOIN bronze.former_names fn_home ON g.home_team = fn_home.former
LEFT JOIN bronze.former_names fn_away ON g.away_team = fn_away.former
WHERE g.scorer IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.goalscorers 
SET TBLPROPERTIES (
  'comment' = 'Registro de gols marcados em partidas, incluindo artilheiros e tipos de gol. Fonte: bronze.goalscorers.'
);

-- Comentários das colunas (um comando por linha)
ALTER TABLE silver.goalscorers CHANGE COLUMN match_date match_date DATE COMMENT 'Data da partida. Domínio: Deve existir em silver.results.match_date.';
ALTER TABLE silver.goalscorers CHANGE COLUMN scoring_team scoring_team STRING COMMENT 'Time que marcou o gol. Domínio: Valores de silver.results.home_team/away_team.';
ALTER TABLE silver.goalscorers CHANGE COLUMN home_team home_team STRING COMMENT 'Time da casa padronizado. Domínio: Valores de silver.results.home_team.';
ALTER TABLE silver.goalscorers CHANGE COLUMN away_team away_team STRING COMMENT 'Time visitante padronizado. Domínio: Valores de silver.results.away_team.';
ALTER TABLE silver.goalscorers CHANGE COLUMN player_name_raw player_name_raw STRING COMMENT 'Nome do jogador sem formatação. Domínio: Texto livre (ex: "messi", "Pelé").';
ALTER TABLE silver.goalscorers CHANGE COLUMN is_own_goal is_own_goal BOOLEAN COMMENT 'Indica se o gol foi contra. Domínio: TRUE/FALSE.';
ALTER TABLE silver.goalscorers CHANGE COLUMN is_penalty is_penalty BOOLEAN COMMENT 'Indica se o gol foi de pênalti. Domínio: TRUE/FALSE.';

In [0]:
%sql
DESCRIBE TABLE silver.goalscorers;

col_name,data_type,comment
match_id,string,null
match_date,date,Data da partida. Domínio: Deve existir em silver.results.match_date.
scoring_team,string,Time que marcou o gol. Domínio: Valores de silver.results.home_team/away_team.
home_team,string,Time da casa padronizado. Domínio: Valores de silver.results.home_team.
away_team,string,Time visitante padronizado. Domínio: Valores de silver.results.away_team.
player_name_raw,string,"Nome do jogador sem formatação. Domínio: Texto livre (ex: ""messi"", ""Pelé"")."
is_own_goal,boolean,Indica se o gol foi contra. Domínio: TRUE/FALSE.
is_penalty,boolean,Indica se o gol foi de pênalti. Domínio: TRUE/FALSE.


É verificada a criação da tabela.

In [0]:
%sql
-- Verificar a tabela silver.goalscorers
SELECT * FROM silver.goalscorers LIMIT 20;

match_id,match_date,scoring_team,home_team,away_team,player_name_raw,is_own_goal,is_penalty
1916-07-02_Chile_Uruguay_0_4,1916-07-02,Uruguay,Chile,Uruguay,José Piendibene,false,false
1916-07-02_Chile_Uruguay_0_4,1916-07-02,Uruguay,Chile,Uruguay,Isabelino Gradín,false,false
1916-07-02_Chile_Uruguay_0_4,1916-07-02,Uruguay,Chile,Uruguay,Isabelino Gradín,false,false
1916-07-02_Chile_Uruguay_0_4,1916-07-02,Uruguay,Chile,Uruguay,José Piendibene,false,false
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Argentina,Chile,Alberto Marcovecchio,false,false
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Argentina,Chile,Alberto Ohaco,false,false
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Argentina,Chile,Alberto Marcovecchio,false,false
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Argentina,Chile,Juan Domingo Brown,false,true
1916-07-06_Argentina_Chile_6_1,1916-07-06,Argentina,Argentina,Chile,Juan Domingo Brown,false,true
1916-07-06_Argentina_Chile_6_1,1916-07-06,Chile,Argentina,Chile,Telésforo Báez,false,false


### Tabela FIFA_Ranking
O fluxo de transformação para criação da tabela silver.fifa_ranking é descrito a seguir:
- Padronização de Datas: Converte rank_date para DATE.

- Unificação de Nomes de Países: Usa bronze.former_names para garantir consistência.

- Conversão de Tipos Numéricos: Converte rank para INT e total_points para DECIMAL(10,2).

In [0]:
%sql
CREATE OR REPLACE TABLE silver.fifa_ranking AS
SELECT 
  date(r.rank_date) AS rank_date,
  COALESCE(fn.current, r.country_full) AS country,
  CAST(r.rank AS INT) AS rank,
  CAST(r.total_points AS DECIMAL(10,2)) AS total_points
FROM bronze.fifa_ranking r
LEFT JOIN bronze.former_names fn ON r.country_full = fn.former;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.fifa_ranking 
SET TBLPROPERTIES (
  'comment' = 'Rankings FIFA históricos de seleções nacionais. Fonte: bronze.fifa_ranking.'
);

-- Comentários das colunas
ALTER TABLE silver.fifa_ranking CHANGE COLUMN rank_date rank_date DATE COMMENT 'Data de publicação do ranking. Domínio: Datas entre 1992-08-17 (primeiro ranking FIFA) e a data atual.';
ALTER TABLE silver.fifa_ranking CHANGE COLUMN country country STRING COMMENT 'Nome do país padronizado. Domínio: Valores de silver.teams.standardized_name.';
ALTER TABLE silver.fifa_ranking CHANGE COLUMN rank rank INT COMMENT 'Posição no ranking FIFA. Domínio: Inteiro entre 1 e 211 (total de associações FIFA).';
ALTER TABLE silver.fifa_ranking CHANGE COLUMN total_points total_points DECIMAL(10, 2) COMMENT 'Pontos totais no ranking. Domínio: Decimal entre 0 e 1842.52 (máximo histórico).';

In [0]:
%sql
DESCRIBE TABLE silver.fifa_ranking;

col_name,data_type,comment
rank_date,date,Data de publicação do ranking. Domínio: Datas entre 1992-08-17 (primeiro ranking FIFA) e a data atual.
country,string,Nome do país padronizado. Domínio: Valores de silver.teams.standardized_name.
rank,int,Posição no ranking FIFA. Domínio: Inteiro entre 1 e 211 (total de associações FIFA).
total_points,"decimal(10,2)",Pontos totais no ranking. Domínio: Decimal entre 0 e 1842.52 (máximo histórico).


É verificada a criação da tabela.

In [0]:
%sql
SELECT *
FROM silver.fifa_ranking LIMIT 20;

rank_date,country,rank,total_points
1992-12-31,Brunei Darussalam,140,2.00
1992-12-31,Portugal,33,38.00
1992-12-31,Zambia,32,38.00
1992-12-31,Greece,31,38.00
1992-12-31,Algeria,30,39.00
1992-12-31,Yugoslavia,29,39.00
1992-12-31,Wales,28,40.00
1992-12-31,Côte d'Ivoire,27,41.00
1992-12-31,Austria,34,38.00
1992-12-31,Bulgaria,26,41.00


Também são realizadas algumas atualizações de nomes dos países para o mesmo nome registrado nas tabelas anteriores.

In [0]:
%sql
-- Atualiza os nomes dos países na tabela silver.ranking para padronização
UPDATE silver.fifa_ranking
SET country = CASE 
    WHEN country = 'Brunei Darussalam' THEN 'Brunei'
    WHEN country = 'Congo DR' THEN 'DR Congo'
    WHEN country = 'Chinese Taipei' THEN 'Taiwan'
    WHEN country = 'Cabo Verde' THEN 'Cape Verde'
    WHEN country = 'Curacao' THEN 'Curaçao'
    WHEN country = 'Czechia' THEN 'Czech Republic'
    WHEN country = 'The Gambia' THEN 'Gambia'
    WHEN country = 'IR Iran' THEN 'Iran'
    WHEN country = "Côte d'Ivoire" THEN 'Ivory Coast'
    WHEN country = 'Kyrgyz Republic' THEN 'Kyrgyzstan'
    WHEN country = 'Korea DPR' THEN 'North Korea'
    WHEN country = 'St Kitts and Nevis' THEN 'Saint Kitts and Nevis'
    WHEN country = 'St Lucia' THEN 'Saint Lucia'
    WHEN country = 'Korea Republic' THEN 'South Korea'
    WHEN country = 'Sao Tome and Principe' THEN 'São Tomé and Príncipe'
    WHEN country = 'USA' THEN 'United States'
    WHEN country = 'St Vincent and the Grenadines' THEN 'Saint Vincent and the Grenadines'
    WHEN country = 'US Virgin Islands' THEN 'United States Virgin Islands'
    ELSE country
END
WHERE country IN (
    'Brunei Darussalam', 'Congo DR', 'Chinese Taipei', 'Cabo Verde', 
    'Curacao', 'Czechia', 'The Gambia', 'IR Iran', "Côte d'Ivoire", 
    'Korea DPR', 'St Kitts and Nevis', 'St Lucia', 'Korea Republic', 
    'Sao Tome and Principe', 'USA', 'Kyrgyz Republic', 
    'St Vincent and the Grenadines', 'US Virgin Islands'
);

num_affected_rows
5594


In [0]:
%sql
SELECT DISTINCT country FROM silver.fifa_ranking
WHERE country IN ('Brunei', 'DR Congo', 'Taiwan', 'Cape Verde', 'Curaçao', 
                      'Czech Republic', 'Gambia', 'Iran', 'Ivory Coast', 
                      'North Korea', 'Saint Kitts and Nevis', 'Saint Lucia', 
                      'South Korea', 'São Tomé and Príncipe','Kyrgyzstan',
                      'Saint Vincent and the Grenadines', 
                      'United States Virgin Islands')
ORDER BY country;

country
Brunei
Cape Verde
Curaçao
Czech Republic
DR Congo
Gambia
Iran
Ivory Coast
Kyrgyzstan
North Korea


Também é verificado se há algum país sem ranking definido na tabela.

In [0]:
%sql
SELECT *
FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216; -- Assumindo que há 211 países no ranking

rank_date,country,rank,total_points
2023-10-26,Eritrea,null,855.56
2023-10-26,Tonga,null,861.81
2023-10-26,Samoa,null,894.26
2023-10-26,American Samoa,null,900.27
2023-11-30,Eritrea,null,855.56
2023-12-21,Eritrea,null,855.56
2024-02-15,Eritrea,null,855.56
2024-04-04,Eritrea,null,855.56
2024-06-20,Eritrea,null,855.56


A partir dessa identificação, é realizada uma correção que exclui essas linhas da tabela.

In [0]:
%sql
DELETE FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216;

num_affected_rows
9


Verificando a exclusão.

In [0]:
%sql
SELECT *
FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216; -- Assumindo que há 216 países no ranking

rank_date,country,rank,total_points


Desse modo, foram construídas as tabelas na camada Silver. A partir disso, as tabelas na camada Gold poderão ser construídas de forma a ser possível responder as perguntas do problema a partir de consultas. O processo de construção da camada Gold está descrito no notebook MVP - Engenharia de Dados - Camada Gold.